Грузим датасет:

In [1]:
cd ..

/home/bfilippov/kaggle/whiteboard


In [3]:
import keras
import pandas as pd
import ast

train = pd.read_csv('source/train.csv')
test = pd.read_csv('source/test.csv')
train['labels'] = train['labels'].map(ast.literal_eval)
test['labels'] = test['labels'].map(ast.literal_eval)

Объявляем модель:

In [4]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf

from keras.layers import Input
from keras import backend as K

from models.model_zoo import lenet_like_convnet, whiteboard_detector, whiteboard_loss
from models.image_generator import whiteboard_images

sess = tf.Session()
K.set_session(sess)

input_height = 128
input_width = 128

input_image = Input(shape=(input_height, input_width, 3))
lenet = lenet_like_convnet(input_image)

simple_detector = whiteboard_detector(input_image, lenet)

In [5]:
from matplotlib import pyplot as plt
from keras.callbacks import ProgbarLogger, TensorBoard, ModelCheckpoint, Callback

model_checkpoint = ModelCheckpoint('lenet_checkpoint', save_best_only=True)
tensor_board = TensorBoard()
callbacks = [model_checkpoint, tensor_board]

In [6]:
optimizer = keras.optimizers.SGD(lr=0.1, momentum=0.5, nesterov=True)
simple_detector.compile(optimizer=optimizer, loss=whiteboard_loss)
# TODO: move momentum up when gradients will be smaller

Учим модель:

In [8]:
img_dir = 'source/'
img_size = (input_height, input_width)

valid_imgs = whiteboard_images(test, img_dir, img_size, batch_size=64)
imgs = whiteboard_images(train, img_dir, img_size, batch_size=128)

simple_detector.fit_generator(imgs, samples_per_epoch=1024, nb_epoch=1000, validation_data=valid_imgs,
                              nb_val_samples=64, callbacks=callbacks)

Epoch 1/1000
1024/1024 [==============================] - 306s - loss: 2.2246 - val_loss: 2.5920
Epoch 2/1000


KeyboardInterrupt: 

In [5]:
valid_imgs = whiteboard_images(test, img_dir, img_size)
valid_batch_input, valid_batch_labels = next(valid_imgs)
predictions = simple_detector.predict_on_batch(valid_batch_input)

Выглядит так что сетка пытается детектировать доску не смотря на её углы:

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import patches

img_ix = 3

plt.imshow(valid_batch_input[img_ix])
plt.plot(valid_batch_labels[img_ix][2:6], valid_batch_labels[img_ix][6:10], '+')
plt.plot(predictions[img_ix][2:6], predictions[img_ix][6:10], 'o')

print("is present?", predictions[img_ix][1])

NameError: name 'valid_batch_input' is not defined

In [7]:
simple_detector.save('')